# Morphing

> This module apply morphing to a template source from landmarks. Morphing from pygem library.


In [ ]:
#| default_exp morph

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import numpy as np
import click
from argparse import ArgumentParser

In [ ]:
#| export
import infepy.rbf as rbf
import infepy.preprocessing as pre
import infepy.utils as utils

In [ ]:
#| export
def morphing(source_landmarks: np.ndarray,  # Landmarks of the source mesh
             target_landmarks: np.ndarray,  # Target Landmarks
             source_mesh: np.ndarray, # Coordinates of the mesh/ mesh 
             func,
             smoothing):
    "Morph the target mesh with RBF function - Thin Plate Spine."
    basis_function = func
    
    rad_bas_fun = rbf.RBF(original_control_points=utils.from_df_to_np(source_landmarks) , 
              deformed_control_points= utils.from_df_to_np(target_landmarks),
              func= basis_function,
              smoothing= smoothing)

    return rad_bas_fun(utils.from_df_to_np(source_mesh))

## CALL from Command Line:
- move into infepy/infepy directory
- type python -m morph

In [ ]:
#| export
parser = ArgumentParser(description="Morphing function")
parser.add_argument("--function", 
                    type=str,
                    default='thin_plate_spline',
                    help="Basis Function for morphing")
parser.add_argument("--smoothing", 
                    type=float,
                    default=0.0,
                    help="Smoothing Factor")
        
def do_morphing(func, smoothing):
    config = utils.read_toml()
    source_landmarks = pre.read_landmarks(utils._merge_path(config['source']['path'],config['source']['filename_landmarks']))
    template_mesh = pre.read_nodes(utils._merge_path(config['source']['path'],config['source']['filename_mesh']))
    
    if utils.multiple_targets() == False: # single target
        target_landmarks = pre.read_landmarks(utils._merge_path(config['target']['path'],config['target']['filename_landmarks']))
        pre._check_landmarks(source_landmarks, target_landmarks)
        morphed_mesh = morphing(source_landmarks, target_landmarks, template_mesh,func, smoothing)
        pre.write_output(morphed_mesh,
                     utils._merge_path(config['target']['path'], 'morphed.key'),
                     utils._merge_path(config['source']['path'], config['source']['filename_mesh']))
    else:
        targets_folder = utils.multiple_targets()
        for folder in targets_folder:
            folder_path = os.path.join(config['target']['path'],folder)
            target_landmarks = pre.read_landmarks(utils._merge_path(folder_path,config['target']['filename_landmarks']))
            pre._check_landmarks(source_landmarks, target_landmarks)
            morphed_mesh = morphing(source_landmarks, target_landmarks, template_mesh,func, smoothing)
            pre.write_output(morphed_mesh,
                      utils._merge_path(folder_path,'morphed.key'),
                      utils._merge_path(config['source']['path'],config['source']['filename_mesh']))
    return

In [ ]:
#| hide
# do_morphing(func='thin_plate_spline', smoothing=None)

In [ ]:
#| export
if __name__ == "__main__":
    args= parser.parse_args()
    print("Radial Basis Function: ", args.function)
    print("Smoothing factor: ", args.smoothing)
    do_morphing(args.function, args.smoothing)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()